<a href="https://colab.research.google.com/github/shiyu-cheng/6895_Visual_Sentiment_Prediction/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch.nn as nn
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms, models, datasets
import torch.optim as optim
import time
import copy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from collections import Counter
from sklearn.metrics import confusion_matrix
import pandas as pd
import itertools
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [ ]:
!unzip '/content/drive/MyDrive/data/zipdata.zip' -d '/content/drive/MyDrive/data/image_data'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
transform = transforms.Compose([
    transforms.Resize(48),        # Resize the image to 256x256 pixels
    #transforms.CenterCrop(224),   # Crop the center 224x224 pixels
    transforms.ToTensor(),        # Convert the image to a PyTorch tensor
    # transforms.Normalize(         # Normalize the image
    #     mean=[0.485, 0.456, 0.406],
    #     std=[0.229, 0.224, 0.225]
    # )
])

data_dir = '/content/drive/MyDrive/Colab Notebooks/image_data'
dataset = datasets.ImageFolder(data_dir, transform=transform)

In [ ]:
total_count=len(dataset)
train_count = int(0.7 * total_count) 
valid_count = total_count - train_count 
train_dataset, valid_dataset = torch.utils.data.random_split(dataset=dataset, lengths=[train_count, valid_count])
batch_size=16
train_dataset_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  
valid_dataset_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False) 
dataloaders = {'train': train_dataset_loader, 'valid': valid_dataset_loader}

In [ ]:
def train_model(model, dataloaders, criterion, optimizer,device, num_epochs=50):
    since = time.time()
    best_acc = 0
    model.to(device)

    val_acc_history = []
    train_acc_history = []

    test_acc_history = []
    train_losses = []
    valid_losses = []
    test_losses = []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

          
            for inputs, labels in dataloaders[phase]:
                #print(inputs.shape)
                inputs = inputs.to(device)
                labels = labels.to(device)

           
                optimizer.zero_grad()
            
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

               
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            
            time_elapsed = time.time() - since
   
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'valid':
                val_acc_history.append(epoch_acc)
                valid_losses.append(epoch_loss)

            if phase == 'train':
                train_acc_history.append(epoch_acc)
                train_losses.append(epoch_loss)
            


    return model, val_acc_history, train_acc_history,valid_losses, train_losses


# define plot accuracy and loss
def plot_acc_loss(val_acc_history, train_acc_history,valid_losses, train_losses):
  val_acc=[]
  for i in val_acc_history:
    val_acc.append(i.cpu().data.numpy())
  train_acc=[]
  for i in train_acc_history:
    train_acc.append(i.cpu().data.numpy())
  test_acc=[]

  
  plt.subplot(1,2,1)
  plt.plot(range(len(val_acc)),val_acc,label='val_acc')
  plt.plot(range(len(train_acc)),train_acc,label='train_acc')
  plt.xlabel('epoch')
  plt.ylabel('acc')
  plt.legend()
  

  plt.subplot(1,2,2)
  plt.plot(range(len(valid_losses)),valid_losses,label='val_loss')
  plt.plot(range(len(train_losses)),train_losses,label='train_loss')
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.legend()
  plt.show()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
from torchvision import models
import torch.nn as nn
model = models.resnet50(pretrained=True)  #Use the resnet18 model
num_ftrs =model .fc.in_features #Modify the number of model categories
model.fc = nn.Sequential(nn.Dropout(0.6),nn.Linear(num_ftrs, 24))


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cost_fun = nn.CrossEntropyLoss().to(device) 
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3) 
model = model.to(device)
model, val_acc_history, train_acc_history,valid_losses, train_losses= train_model(model, dataloaders, cost_fun, optimizer, device,num_epochs=100)

Epoch 0/99
----------
train Loss: 3.3157 Acc: 0.0686
valid Loss: 3.1492 Acc: 0.0675
Epoch 1/99
----------
train Loss: 3.2197 Acc: 0.0784
valid Loss: 3.1048 Acc: 0.0931
Epoch 2/99
----------
train Loss: 3.1771 Acc: 0.0896
valid Loss: 3.5922 Acc: 0.1019
Epoch 3/99
----------
train Loss: 3.1407 Acc: 0.0989
valid Loss: 3.3183 Acc: 0.1237
Epoch 4/99
----------
train Loss: 3.1299 Acc: 0.1036
valid Loss: 38.4826 Acc: 0.0872
Epoch 5/99
----------
train Loss: 3.1371 Acc: 0.1037
valid Loss: 4.5954 Acc: 0.1043
Epoch 6/99
----------
train Loss: 3.0904 Acc: 0.1195
valid Loss: 2.9814 Acc: 0.1296
Epoch 7/99
----------
train Loss: 3.0682 Acc: 0.1158
valid Loss: 2.9830 Acc: 0.1429
Epoch 8/99
----------
train Loss: 3.0484 Acc: 0.1254
valid Loss: 3.4241 Acc: 0.1509
Epoch 9/99
----------
train Loss: 3.0241 Acc: 0.1292
valid Loss: 2.9230 Acc: 0.1585
Epoch 10/99
----------
train Loss: 2.9905 Acc: 0.1415
valid Loss: 3.2174 Acc: 0.1400
Epoch 11/99
----------
train Loss: 2.9747 Acc: 0.1412
valid Loss: 3.4602 A

KeyboardInterrupt: ignored